In [77]:
# https://github.com/lingjzhu/clap-ipa/tree/main
import librosa
import numpy as np
clap_ipa_sr = 16000
some_audio,sampling_rate = librosa.load("tutorials/audio-samples/angelo-reloading.mp3")
some_ipa_string = "ˌriːˈləʊdɪŋ"
some_audio = librosa.resample(some_audio, orig_sr=sampling_rate, target_sr=16000)
sampling_rate = 16000
some_audio

array([-8.9143455e-08,  2.3081616e-06,  4.6085488e-06, ...,
        2.3047534e-05, -2.9918798e-05, -7.1172952e-05], dtype=float32)

In [78]:
import sys 
sys.path.append("./clap-ipa")
from clap.encoders import *
import torch.nn.functional as F
from transformers import DebertaV2Tokenizer, AutoProcessor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

speech_encoder = SpeechEncoder.from_pretrained('anyspeech/clap-ipa-tiny-speech')
phone_encoder = PhoneEncoder.from_pretrained('anyspeech/clap-ipa-tiny-phone')
phone_encoder.eval().to(device)
speech_encoder.eval().to(device)

tokenizer = DebertaV2Tokenizer.from_pretrained('charsiu/IPATokenizer')
processor = AutoProcessor.from_pretrained('openai/whisper-tiny')

import time
start = time.time()
audio_input = processor(some_audio,return_attention_mask=True,sampling_rate=16000,max_length=32000,return_tensors='pt')
ipa_input = tokenizer(some_ipa_string,return_token_type_ids=False, return_tensors='pt')
with torch.no_grad():
   speech_embed = speech_encoder(**audio_input).pooler_output
   phone_embed = phone_encoder(**ipa_input).pooler_output

similarity = F.cosine_similarity(speech_embed,phone_embed,dim=-1)
print(time.time()-start)
similarity

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0.020133018493652344


tensor([0.0865])